In [1]:
import xarray as xr
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()

%load_ext autoreload
%autoreload 2

### Paths input data

In [2]:
from ar6_ch6_rcmipfigs.constants import OUTPUT_DATA_DIR, RESULTS_DIR

# PATH_DATASET = OUTPUT_DATA_DIR + '/forcing_data_rcmip_models.nc'
# PATH_DT = OUTPUT_DATA_DIR / '/dT_data_rcmip_models.nc'
PATH_DT = OUTPUT_DATA_DIR / 'dT_data_RCMIP_recommendation.nc'

/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs
/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs/data_in


#### Uncertainty data from Chris

In [3]:
PATH_DT_UNCERTAINTY = OUTPUT_DATA_DIR / 'dT_uncertainty_data_FaIR_chris.nc'

## Set values:

In [4]:
first_y = 1750
last_y = 2100

**Set reference year for temperature change:**

In [5]:
ref_year = 2020

In [6]:
FIGURE_DIR = RESULTS_DIR / 'figures_recommendation/'

TABLE_DIR = RESULTS_DIR / 'tables_recommendation/'

In [7]:
from pathlib import Path
Path(FIGURE_DIR).mkdir(parents=True, exist_ok=True)
Path(TABLE_DIR).mkdir(parents=True, exist_ok=True)

In [8]:
percentile = 'percentile'
climatemodel = 'climatemodel'
scenario = 'scenario'
variable = 'variable'
time = 'time'
name_deltaT = 'Delta T'


### Define variables to look at:

In [9]:
# variables to plot:
variables_erf_comp = [
    'ch4',
    # 'aerosol-radiation_interactions',
    # 'aerosol-cloud_interactions',
    'aerosol-total',
    'o3',
    'HFCs',
    'bc_on_snow']
# total ERFs for anthropogenic and total:
#variables_erf_tot = ['total_anthropogenic',
#                     'total']

scenarios_fl_370 = ['ssp370', 'ssp370-lowNTCF-aerchemmip', 'ssp370-lowNTCF-gidden'  # Due to mistake here
                    ]

### Scenarios:

In [10]:
scenarios_fl = ['ssp119',
                'ssp126',
                'ssp245',
                'ssp370',
                'ssp370-lowNTCF-aerchemmip',
                #'ssp370-lowNTCF-gidden',
                'ssp370-lowNTCF-gidden',
                'ssp585']

In [11]:
median = 'median'
perc5 = '5th percentile'
perc95 = '95th percentile'
recommendation = 'recommendation'

## Open dataset:

### Integrate:
The code below opens the file generated in [2_compute_delta_T.ipynb](2_compute_delta_T.ipynb) by integrating

\begin{align*} 
\Delta T (t) &= \int_0^t ERF(t') IRF(t-t') dt' \\
\end{align*}

where IRF is the impulse response function and ERF is the effective radiative forcing from RCMIP. 

In [12]:
ds_DT = xr.open_dataset(PATH_DT)
ds_uncertainty = xr.open_dataset(PATH_DT_UNCERTAINTY)

In [13]:
for var in variables_erf_comp:
    da5 = ds_uncertainty.sel(variable=var, scenario='ssp585')['p05-p50']    
    da95 = ds_uncertainty.sel(variable=var, scenario='ssp585')['p95-p50']
    da5.plot(label=var)
    da95.plot(label=var)

## Merge uncertainty and original: 


In [14]:
percentiles_to_keep = ['p05-p50','p16-p50','p84-p50','p95-p50']

In [15]:
_da = ds_uncertainty[percentiles_to_keep].to_array('percentile')
_ds = _da.rename(name_deltaT).to_dataset()

In [16]:
ds_DT= xr.concat([ds_DT[name_deltaT],_ds[name_deltaT]], dim='percentile').to_dataset()

In [17]:
ds_DT

<xarray.Dataset>
Dimensions:      (base_period: 1, percentile: 5, scenario: 10, variable: 20, year: 351)
Coordinates:
  * year         (year) int64 1750 1751 1752 1753 1754 ... 2097 2098 2099 2100
  * variable     (variable) object 'HFCs' ... 'volcanic'
  * scenario     (scenario) object 'ssp119' 'ssp126' ... 'ssp534-over' 'ssp585'
  * base_period  (base_period) int64 2020
  * percentile   (percentile) object 'recommendation' 'p05-p50' ... 'p95-p50'
Data variables:
    Delta T      (percentile, scenario, variable, year, base_period) float64 0.0 ... nan

## Make csv table of total values:

In [18]:
from ar6_ch6_rcmipfigs.utils.plot import get_var_nicename

<IPython.core.display.Javascript object>

In [19]:
reference_year = ref_year
start_y_tabel = 2015
end_y_tabel = last_y

In [20]:
def get_fn(var_name, s_y, e_y, ref_y, perc):
    _st = var_name.replace('(','').replace(')','').replace(' ','_').replace(',','')#+'.csv'
    fn = f'{perc}_{_st}_{s_y}-{e_y}_refyear{ref_y}.csv'
    return fn

def make_sum_slcfs_tabel(percentile=recommendation):
    _str = ''
    _vl = [get_var_nicename(var).split('(')[0].strip() for var in variables_erf_comp]
    for var in _vl:
        _str += f'{var}, '
    vn_sum = 'Sum SLCF (%s)' % _str[:-2]

    fn = get_fn(vn_sum, start_y_tabel, end_y_tabel, ref_year, percentile)



    # ref year value:
    ds_DT_sy = ds_DT[name_deltaT].sel(
        variable=variables_erf_comp,
        year=reference_year
    ).sum(variable).squeeze()

    # all values from s_y to e_y
    _da = ds_DT[name_deltaT].sel(
        variable=variables_erf_comp,
        year=slice(start_y_tabel, end_y_tabel)
    ).sum(variable).sel(year=slice(int(start_y_tabel), int(end_y_tabel))) - ds_DT_sy

    # Choose recommendation::
    _pl_da = _da.sel(percentile=percentile).squeeze()
    df = _pl_da.to_pandas().transpose()
    df['percentile'] = percentile
    display(df)
    fn = TABLE_DIR / fn
    df.to_csv(fn)
    df
for prc in [recommendation, 'p05-p50','p95-p50']:
    make_sum_slcfs_tabel(percentile=prc)
    print(prc)

scenario,ssp119,ssp126,ssp245,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,
2015,-0.079958,-0.078379,-0.079069,-0.066064,-0.070390,-0.070031,-0.074390,-0.074390,-0.090014,-0.089808,recommendation
2016,-0.060279,-0.058699,-0.059390,-0.046385,-0.050711,-0.050351,-0.054711,-0.054711,-0.070335,-0.070129,recommendation
2017,-0.042305,-0.040725,-0.041416,-0.028410,-0.032736,-0.032377,-0.036737,-0.036737,-0.052361,-0.052154,recommendation
2018,-0.027593,-0.026014,-0.026704,-0.013699,-0.018025,-0.017665,-0.022025,-0.022025,-0.037649,-0.037443,recommendation
2019,-0.014042,-0.012462,-0.013152,-0.000147,-0.004473,-0.004114,-0.008473,-0.008473,-0.024097,-0.023891,recommendation
...,...,...,...,...,...,...,...,...,...,...,...
2096,0.104696,0.127281,0.287242,0.550761,0.783495,0.134214,0.414939,0.485862,0.081640,0.646932,recommendation
2097,0.105663,0.127870,0.288571,0.558942,0.791704,0.137475,0.417470,0.487455,0.079856,0.648757,recommendation
2098,0.106707,0.128520,0.289886,0.567137,0.799902,0.140799,0.419997,0.489028,0.078213,0.650552,recommendation


recommendation


scenario,ssp119,ssp126,ssp245,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,
2015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p05-p50
2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p05-p50
2017,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p05-p50
2018,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p05-p50
2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.425139,-0.410495,-0.288934,-0.321367,-0.361241,-0.332515,-0.226323,-0.248893,-0.413024,-0.307703,p05-p50
2097,-0.427212,-0.414072,-0.292618,-0.323899,-0.364968,-0.334091,-0.229946,-0.251286,-0.415271,-0.310496,p05-p50
2098,-0.429237,-0.417831,-0.296129,-0.326121,-0.368645,-0.335480,-0.233863,-0.253295,-0.417183,-0.312715,p05-p50


p05-p50


scenario,ssp119,ssp126,ssp245,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,
2015,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p95-p50
2016,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p95-p50
2017,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p95-p50
2018,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p95-p50
2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,p95-p50
...,...,...,...,...,...,...,...,...,...,...,...
2096,0.516753,0.493755,0.411266,0.344383,0.524262,0.400145,0.315806,0.369100,0.497889,0.464328,p95-p50
2097,0.519737,0.498523,0.417207,0.348250,0.530969,0.403649,0.320138,0.373015,0.499577,0.467065,p95-p50
2098,0.522073,0.502969,0.421522,0.351573,0.537044,0.406753,0.325402,0.377089,0.500839,0.470289,p95-p50


p95-p50


In [21]:
import pandas as pd

## TABLE EACH VAR

In [22]:

def make_slcfs_tabel(var, percentile=recommendation):
    _str = ''
    _st = get_fn(var, start_y_tabel, end_y_tabel, ref_year, percentile)


    ds_DT_sy = ds_DT[name_deltaT].sel(
        variable=var, 
        year=reference_year,
    ).squeeze()
    
    _da = ds_DT[name_deltaT].sel(
        variable=var,
        year=slice(start_y_tabel, end_y_tabel)) - ds_DT_sy
    
    # Take recommendation::
    _pl_da = _da.sel(percentile=recommendation).squeeze()

    df = _pl_da.to_pandas().transpose()
    df['percentile'] = percentile
    display(df)
    fn = TABLE_DIR/_st
    df.to_csv(fn)

for var in variables_erf_comp:
    for prc in [recommendation, 'p05-p50','p95-p50']:
        make_slcfs_tabel(var, percentile=prc)

scenario,ssp119,ssp126,ssp245,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,
2015,-0.010446,-0.010175,-0.011257,-0.011757,-0.011757,-0.011543,-0.012349,-0.012349,-0.011188,-0.011061,recommendation
2016,-0.008579,-0.008308,-0.009391,-0.009890,-0.009890,-0.009677,-0.010482,-0.010482,-0.009322,-0.009194,recommendation
2017,-0.006735,-0.006464,-0.007546,-0.008046,-0.008046,-0.007832,-0.008638,-0.008638,-0.007477,-0.007350,recommendation
2018,-0.004837,-0.004566,-0.005649,-0.006148,-0.006148,-0.005935,-0.006740,-0.006740,-0.005580,-0.005452,recommendation
2019,-0.002899,-0.002628,-0.003710,-0.004210,-0.004210,-0.003996,-0.004802,-0.004802,-0.003641,-0.003514,recommendation
...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.157716,-0.149590,0.000742,0.265116,0.265116,-0.124106,0.125782,0.178841,-0.161917,0.153667,recommendation
2097,-0.159108,-0.151092,0.000247,0.268031,0.268031,-0.123713,0.125689,0.178938,-0.163362,0.152101,recommendation
2098,-0.160467,-0.152549,-0.000213,0.270925,0.270925,-0.123291,0.125559,0.179021,-0.164730,0.150545,recommendation


scenario,ssp119,ssp126,ssp245,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,
2015,-0.010446,-0.010175,-0.011257,-0.011757,-0.011757,-0.011543,-0.012349,-0.012349,-0.011188,-0.011061,p05-p50
2016,-0.008579,-0.008308,-0.009391,-0.009890,-0.009890,-0.009677,-0.010482,-0.010482,-0.009322,-0.009194,p05-p50
2017,-0.006735,-0.006464,-0.007546,-0.008046,-0.008046,-0.007832,-0.008638,-0.008638,-0.007477,-0.007350,p05-p50
2018,-0.004837,-0.004566,-0.005649,-0.006148,-0.006148,-0.005935,-0.006740,-0.006740,-0.005580,-0.005452,p05-p50
2019,-0.002899,-0.002628,-0.003710,-0.004210,-0.004210,-0.003996,-0.004802,-0.004802,-0.003641,-0.003514,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.157716,-0.149590,0.000742,0.265116,0.265116,-0.124106,0.125782,0.178841,-0.161917,0.153667,p05-p50
2097,-0.159108,-0.151092,0.000247,0.268031,0.268031,-0.123713,0.125689,0.178938,-0.163362,0.152101,p05-p50
2098,-0.160467,-0.152549,-0.000213,0.270925,0.270925,-0.123291,0.125559,0.179021,-0.164730,0.150545,p05-p50


scenario,ssp119,ssp126,ssp245,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,
2015,-0.010446,-0.010175,-0.011257,-0.011757,-0.011757,-0.011543,-0.012349,-0.012349,-0.011188,-0.011061,p95-p50
2016,-0.008579,-0.008308,-0.009391,-0.009890,-0.009890,-0.009677,-0.010482,-0.010482,-0.009322,-0.009194,p95-p50
2017,-0.006735,-0.006464,-0.007546,-0.008046,-0.008046,-0.007832,-0.008638,-0.008638,-0.007477,-0.007350,p95-p50
2018,-0.004837,-0.004566,-0.005649,-0.006148,-0.006148,-0.005935,-0.006740,-0.006740,-0.005580,-0.005452,p95-p50
2019,-0.002899,-0.002628,-0.003710,-0.004210,-0.004210,-0.003996,-0.004802,-0.004802,-0.003641,-0.003514,p95-p50
...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.157716,-0.149590,0.000742,0.265116,0.265116,-0.124106,0.125782,0.178841,-0.161917,0.153667,p95-p50
2097,-0.159108,-0.151092,0.000247,0.268031,0.268031,-0.123713,0.125689,0.178938,-0.163362,0.152101,p95-p50
2098,-0.160467,-0.152549,-0.000213,0.270925,0.270925,-0.123291,0.125559,0.179021,-0.164730,0.150545,p95-p50


scenario,ssp119,ssp126,ssp245,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,
2015,-0.061467,-0.060659,-0.055500,-0.035249,-0.042932,-0.042932,-0.045397,-0.045397,-0.063387,-0.063387,recommendation
2016,-0.045877,-0.045068,-0.039910,-0.019658,-0.027342,-0.027342,-0.029806,-0.029806,-0.047797,-0.047797,recommendation
2017,-0.031311,-0.030503,-0.025344,-0.005093,-0.012776,-0.012776,-0.015241,-0.015241,-0.033231,-0.033231,recommendation
2018,-0.021018,-0.020210,-0.015051,0.005200,-0.002483,-0.002483,-0.004948,-0.004948,-0.022938,-0.022938,recommendation
2019,-0.012627,-0.011819,-0.006660,0.013591,0.005908,0.005908,0.003443,0.003443,-0.014548,-0.014548,recommendation
...,...,...,...,...,...,...,...,...,...,...,...
2096,0.454619,0.461298,0.307650,-0.047572,0.303856,0.303856,0.271037,0.186736,0.398856,0.239354,recommendation
2097,0.457472,0.464383,0.310291,-0.045808,0.305918,0.305918,0.275252,0.190314,0.399518,0.244881,recommendation
2098,0.460336,0.467446,0.312850,-0.044011,0.307988,0.307988,0.279495,0.193847,0.400166,0.250463,recommendation


scenario,ssp119,ssp126,ssp245,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,
2015,-0.061467,-0.060659,-0.055500,-0.035249,-0.042932,-0.042932,-0.045397,-0.045397,-0.063387,-0.063387,p05-p50
2016,-0.045877,-0.045068,-0.039910,-0.019658,-0.027342,-0.027342,-0.029806,-0.029806,-0.047797,-0.047797,p05-p50
2017,-0.031311,-0.030503,-0.025344,-0.005093,-0.012776,-0.012776,-0.015241,-0.015241,-0.033231,-0.033231,p05-p50
2018,-0.021018,-0.020210,-0.015051,0.005200,-0.002483,-0.002483,-0.004948,-0.004948,-0.022938,-0.022938,p05-p50
2019,-0.012627,-0.011819,-0.006660,0.013591,0.005908,0.005908,0.003443,0.003443,-0.014548,-0.014548,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...
2096,0.454619,0.461298,0.307650,-0.047572,0.303856,0.303856,0.271037,0.186736,0.398856,0.239354,p05-p50
2097,0.457472,0.464383,0.310291,-0.045808,0.305918,0.305918,0.275252,0.190314,0.399518,0.244881,p05-p50
2098,0.460336,0.467446,0.312850,-0.044011,0.307988,0.307988,0.279495,0.193847,0.400166,0.250463,p05-p50


...

scenario,ssp119,ssp126,ssp245,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,
2015,-0.005898,-0.005880,-0.005981,-0.005820,-0.005820,-0.005827,-0.005541,-0.005541,-0.005955,-0.005955,p05-p50
2016,-0.004859,-0.004841,-0.004942,-0.004781,-0.004781,-0.004788,-0.004502,-0.004502,-0.004916,-0.004916,p05-p50
2017,-0.003748,-0.003730,-0.003831,-0.003671,-0.003671,-0.003677,-0.003392,-0.003392,-0.003806,-0.003806,p05-p50
2018,-0.002572,-0.002554,-0.002655,-0.002495,-0.002495,-0.002501,-0.002216,-0.002216,-0.002630,-0.002630,p05-p50
2019,-0.001338,-0.001320,-0.001421,-0.001261,-0.001261,-0.001267,-0.000982,-0.000982,-0.001396,-0.001396,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.003930,-0.001362,0.056456,0.092780,0.092780,0.080630,-0.007311,0.045504,0.017160,0.216693,p05-p50
2097,-0.003951,-0.001443,0.056769,0.093866,0.093866,0.081422,-0.007375,0.044357,0.016626,0.218624,p05-p50
2098,-0.003973,-0.001523,0.057064,0.094951,0.094951,0.082214,-0.007437,0.043225,0.016113,0.220448,p05-p50


scenario,ssp119,ssp126,ssp245,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,
2015,-0.005898,-0.005880,-0.005981,-0.005820,-0.005820,-0.005827,-0.005541,-0.005541,-0.005955,-0.005955,p95-p50
2016,-0.004859,-0.004841,-0.004942,-0.004781,-0.004781,-0.004788,-0.004502,-0.004502,-0.004916,-0.004916,p95-p50
2017,-0.003748,-0.003730,-0.003831,-0.003671,-0.003671,-0.003677,-0.003392,-0.003392,-0.003806,-0.003806,p95-p50
2018,-0.002572,-0.002554,-0.002655,-0.002495,-0.002495,-0.002501,-0.002216,-0.002216,-0.002630,-0.002630,p95-p50
2019,-0.001338,-0.001320,-0.001421,-0.001261,-0.001261,-0.001267,-0.000982,-0.000982,-0.001396,-0.001396,p95-p50
...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.003930,-0.001362,0.056456,0.092780,0.092780,0.080630,-0.007311,0.045504,0.017160,0.216693,p95-p50
2097,-0.003951,-0.001443,0.056769,0.093866,0.093866,0.081422,-0.007375,0.044357,0.016626,0.218624,p95-p50
2098,-0.003973,-0.001523,0.057064,0.094951,0.094951,0.082214,-0.007437,0.043225,0.016113,0.220448,p95-p50


scenario,ssp119,ssp126,ssp245,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,
2015,0.000378,0.000458,-0.001487,-0.004118,-0.003045,-0.003045,-0.003694,-0.003694,-0.002600,-0.002600,recommendation
2016,0.000247,0.000327,-0.001618,-0.004249,-0.003176,-0.003176,-0.003825,-0.003825,-0.002731,-0.002731,recommendation
2017,0.000120,0.000200,-0.001745,-0.004376,-0.003303,-0.003303,-0.003952,-0.003952,-0.002858,-0.002858,recommendation
2018,-0.000030,0.000051,-0.001895,-0.004526,-0.003452,-0.003452,-0.004101,-0.004101,-0.003008,-0.003008,recommendation
2019,-0.000218,-0.000137,-0.002083,-0.004714,-0.003641,-0.003641,-0.004290,-0.004290,-0.003196,-0.003196,recommendation
...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.041158,-0.038601,-0.034686,0.014365,-0.026366,-0.026366,-0.024978,-0.020348,-0.040415,-0.026463,recommendation
2097,-0.041249,-0.038802,-0.035012,0.014185,-0.026630,-0.026630,-0.025627,-0.020909,-0.040632,-0.027254,recommendation
2098,-0.041341,-0.039002,-0.035329,0.014003,-0.026895,-0.026895,-0.026274,-0.021465,-0.040843,-0.028050,recommendation


scenario,ssp119,ssp126,ssp245,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,
2015,0.000378,0.000458,-0.001487,-0.004118,-0.003045,-0.003045,-0.003694,-0.003694,-0.002600,-0.002600,p05-p50
2016,0.000247,0.000327,-0.001618,-0.004249,-0.003176,-0.003176,-0.003825,-0.003825,-0.002731,-0.002731,p05-p50
2017,0.000120,0.000200,-0.001745,-0.004376,-0.003303,-0.003303,-0.003952,-0.003952,-0.002858,-0.002858,p05-p50
2018,-0.000030,0.000051,-0.001895,-0.004526,-0.003452,-0.003452,-0.004101,-0.004101,-0.003008,-0.003008,p05-p50
2019,-0.000218,-0.000137,-0.002083,-0.004714,-0.003641,-0.003641,-0.004290,-0.004290,-0.003196,-0.003196,p05-p50
...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.041158,-0.038601,-0.034686,0.014365,-0.026366,-0.026366,-0.024978,-0.020348,-0.040415,-0.026463,p05-p50
2097,-0.041249,-0.038802,-0.035012,0.014185,-0.026630,-0.026630,-0.025627,-0.020909,-0.040632,-0.027254,p05-p50
2098,-0.041341,-0.039002,-0.035329,0.014003,-0.026895,-0.026895,-0.026274,-0.021465,-0.040843,-0.028050,p05-p50


scenario,ssp119,ssp126,ssp245,ssp370,ssp370-lowNTCF-aerchemmip,ssp370-lowNTCF-gidden,ssp434,ssp460,ssp534-over,ssp585,percentile
year,,,,,,,,,,,
2015,0.000378,0.000458,-0.001487,-0.004118,-0.003045,-0.003045,-0.003694,-0.003694,-0.002600,-0.002600,p95-p50
2016,0.000247,0.000327,-0.001618,-0.004249,-0.003176,-0.003176,-0.003825,-0.003825,-0.002731,-0.002731,p95-p50
2017,0.000120,0.000200,-0.001745,-0.004376,-0.003303,-0.003303,-0.003952,-0.003952,-0.002858,-0.002858,p95-p50
2018,-0.000030,0.000051,-0.001895,-0.004526,-0.003452,-0.003452,-0.004101,-0.004101,-0.003008,-0.003008,p95-p50
2019,-0.000218,-0.000137,-0.002083,-0.004714,-0.003641,-0.003641,-0.004290,-0.004290,-0.003196,-0.003196,p95-p50
...,...,...,...,...,...,...,...,...,...,...,...
2096,-0.041158,-0.038601,-0.034686,0.014365,-0.026366,-0.026366,-0.024978,-0.020348,-0.040415,-0.026463,p95-p50
2097,-0.041249,-0.038802,-0.035012,0.014185,-0.026630,-0.026630,-0.025627,-0.020909,-0.040632,-0.027254,p95-p50
2098,-0.041341,-0.039002,-0.035329,0.014003,-0.026895,-0.026895,-0.026274,-0.021465,-0.040843,-0.028050,p95-p50
